In [4]:
from tkinter import *
import random
import time

class Snake(Tk):
    def __init__(self, *args, **kwargs):
        global highest_score
        Tk.__init__(self, *args, **kwargs)
        self.game_over_text = None
        self.initialSetup()
    with open("a.txt", "r") as f:
        highest_score = int(f.read())
        

    def initialSetup(self):
        global score, highest_score

        self.base = Canvas(self, width=1900, height=900, bg="lightgreen")  # Use 'bg' for background color
        self.base.pack(padx=10, pady=10)
        self.snake = self.base.create_rectangle(1, 1, 21, 21, fill="DodgerBlue2")
        self.score = 0
        
        self.scoreDisplay = Label(self, text="Score: {}\n Highest Score: {}".format(self.score, self.highest_score), font=('arial', 20, 'bold'))
        self.scoreDisplay.pack(anchor='n')

        # Add a restart button
        self.restartButton = Button(self, text="Restart Game", font=('arial', 20, 'bold'), command=self.restart)
        self.restartButton.pack(anchor='n')

        self.length = 3
        self.target = None
        self.gameStatus = 1
        self.x = 20
        self.y = 0
        self.bodycoords = [(0, 0)]
        self.bind('<Any-KeyPress>', self.linkKeys)

    def check_snake_coords(self):
        self.base.move(self.snake, self.x, self.y)
        i, j, ii, jj = self.base.coords(self.snake)
        if i <= 0 or j <= 0 or ii >= 1900 or jj >= 900:
            self.x = 0
            self.y = 0
            # Game over
            self.game_over_text = self.base.create_text(950, 450, text="GAME OVER", font=('arial', 70, 'bold'), fill='red')
            self.gameStatus = 0
    def move_snake(self):
        i, j, ii, jj = self.base.coords(self.snake)
        ii = (ii - ((ii - i) / 2))
        jj = (jj - ((jj - j) / 2))
        self.bodycoords.append((ii, jj))
        
        # Check if the snake collides with its own body
        if len(self.bodycoords) > self.length:
            self.base.delete('snakebody')
            del self.bodycoords[0]

        # Check for collisions with the rest of the body
        for segment in self.bodycoords[:-1]:
            if (ii, jj) == segment:
                self.x = 0
                self.y = 0
                # Game over
                self.game_over_text = self.base.create_text(950, 450, text="GAME OVER", font=('arial', 70, 'bold'), fill='red')
                self.gameStatus = 0

        self.base.create_line(tuple(self.bodycoords), tag='snakebody', width=20, fill="DodgerBlue2")
    def food(self):
        if self.target is None:
            a = random.randint(20, 1900)
            b = random.randint(20, 900)
            self.target = self.base.create_oval(a, b, a + 20, b + 20, fill='red', tag='food')

        if self.target:

            i, j, ii, jj = self.base.coords(self.target)

            if len(self.base.find_overlapping(i, j, ii, jj)) != 1:
                self.base.delete("food")
                self.target = None
                self.updateScore()
                self.length += 1
                
                

    def updateScore(self):
        self.score += 1
        if self.score >= self.highest_score:
            self.highest_score = self.score
            with open("a.txt", "w") as f:
                f.write(str(self.highest_score))
        self.scoreDisplay['text'] = "Score: {}\nhighest score:{}".format(self.score,self.highest_score)

    def linkKeys(self, event=None):
        pressedkey = event.keysym
        if pressedkey == 'Left':
            self.x = -20
            self.y = 0
        elif pressedkey == 'Up':
            self.x = 0
            self.y = -20
        elif pressedkey == 'Right':
            self.x = 20
            self.y = 0
        elif pressedkey == 'Down':
            self.x = 0
            self.y = 20
        else:
            pass

    def restart(self):
        # Reset the game state
        if self.game_over_text is not None:
            self.base.delete(self.game_over_text)
        self.base.delete("food")
        self.target = None
        self.gameStatus = 1
        self.x = 20
        self.y = 0
        self.bodycoords = [(0, 0)]
        self.score = 0
        self.scoreDisplay['text'] = "Score: {}\n highest score: {}".format(self.score, self.highest_score)
        self.length = 3
        self.base.delete("snakebody")
        self.base.coords(self.snake, 1, 1, 21, 21)

    def manage(self):
        if self.gameStatus == 0:
            return
        self.check_snake_coords()
        self.move_snake()
        self.food()

snakeobj = Snake(className="ProjectGurukul Snake Game")
with open("a.txt", "r") as f:
    highest_score = int(f.read())
while True:

    snakeobj.update()
    snakeobj.update_idletasks()
    snakeobj.manage()
    time.sleep(0.05)

# Çalışan checkpoint

In [2]:
from tkinter import *
import random

GAME_WIDTH = 1900
GAME_HEIGHT = 925
SPEED = 50
SPACE_SIZE = 25
BODY_PARTS = 3
SNAKE_COLOR = "#00FF00"
FOOD_COLOR = "#FF0000"
BACKGROUND_COLOR = "#000000"


class Snake:

    def __init__(self):
        self.body_size = BODY_PARTS
        self.coordinates = []
        self.squares = []

        for i in range(0, BODY_PARTS):
            self.coordinates.append([0, 0])

        for x, y in self.coordinates:
            square = canvas.create_rectangle(x, y, x + SPACE_SIZE, y + SPACE_SIZE, fill=SNAKE_COLOR, tag="snake")
            self.squares.append(square)


class Food:

    def __init__(self):

        x = random.randint(0, (GAME_WIDTH / SPACE_SIZE)-1) * SPACE_SIZE
        y = random.randint(0, (GAME_HEIGHT / SPACE_SIZE) - 1) * SPACE_SIZE

        self.coordinates = [x, y]

        canvas.create_oval(x, y, x + SPACE_SIZE, y + SPACE_SIZE, fill=FOOD_COLOR, tag="food")


def next_turn(snake, food):

    x, y = snake.coordinates[0]

    if direction == "up":
        y -= SPACE_SIZE
    elif direction == "down":
        y += SPACE_SIZE
    elif direction == "left":
        x -= SPACE_SIZE
    elif direction == "right":
        x += SPACE_SIZE

    snake.coordinates.insert(0, (x, y))

    square = canvas.create_rectangle(x, y, x + SPACE_SIZE, y + SPACE_SIZE, fill=SNAKE_COLOR)

    snake.squares.insert(0, square)

    if x == food.coordinates[0] and y == food.coordinates[1]:
    
        global score
        global highest_score
        score += 1
        if score >= highest_score:
            highest_score = score

            with open("a.txt", "w") as f:
                f.write(str(highest_score))
        label.config(text="Score:{}\nHighest Score:{}".format(score, highest_score))
        canvas.delete("food")
        food = Food()



    else:

        del snake.coordinates[-1]

        canvas.delete(snake.squares[-1])

        del snake.squares[-1]

    if check_collisions(snake):
        game_over()

    else:
        window.after(SPEED, next_turn, snake, food)


def change_direction(new_direction):

    global direction

    if new_direction == 'left':
        if direction != 'right':
            direction = new_direction
    elif new_direction == 'right':
        if direction != 'left':
            direction = new_direction
    elif new_direction == 'up':
        if direction != 'down':
            direction = new_direction
    elif new_direction == 'down':
        if direction != 'up':
            direction = new_direction


def check_collisions(snake):

    x, y = snake.coordinates[0]

    if x < 0 or x >= GAME_WIDTH:
        return True
    elif y < 0 or y >= GAME_HEIGHT:
        return True

    for body_part in snake.coordinates[1:]:
        if x == body_part[0] and y == body_part[1]:
            return True

    return False


def game_over():

    canvas.delete(ALL)
    canvas.create_text(canvas.winfo_width()/2, canvas.winfo_height()/2,
                       font=('consolas',70), text="GAME OVER", fill="red", tag="gameover")

    

window = Tk()
window.title("Snake game")
window.resizable(False, False)



score = 0
highest_score=0
direction = 'down'

with open("a.txt", "r") as f:
    highest_score = int(f.read())

label = Label(window, text="Score:{}\nHighest Score:{}".format(score, highest_score), font=('consolas', 20))
label.pack()

canvas = Canvas(window, bg=BACKGROUND_COLOR, height=GAME_HEIGHT, width=GAME_WIDTH)
canvas.pack()

window.update()

window_width = window.winfo_width()
window_height = window.winfo_height()
screen_width = window.winfo_screenwidth()
screen_height = window.winfo_screenheight()

x = int((screen_width/2) - (window_width/2))
y = int((screen_height/2) - (window_height/2))

window.geometry(f"{window_width}x{window_height}+{x}+{y}")

window.bind('<Left>', lambda event: change_direction('left'))
window.bind('<Right>', lambda event: change_direction('right'))
window.bind('<Up>', lambda event: change_direction('up'))
window.bind('<Down>', lambda event: change_direction('down'))

snake = Snake()
food = Food()

next_turn(snake, food)

window.mainloop()